<a href="https://colab.research.google.com/github/mahb97/Wake2vec/blob/main/heartbeat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import json, pathlib
DRIVE = pathlib.Path("/content/drive/MyDrive/wake2vec")
run = max((DRIVE/"runs").glob("t4_*"), key=lambda p: p.stat().st_mtime)
log = json.loads((run/"metrics"/"phase1_loss_log.json").read_text())
print(run.name, "step", log[-1]["step"], "loss", round(float(log[-1]["loss"]),4))

t4_1762376560 step 550 loss 3.2604


In [4]:
import json, pathlib
DRIVE = pathlib.Path("/content/drive/MyDrive/wake2vec")
run = max((DRIVE/"runs").glob("t4_*"), key=lambda p: p.stat().st_mtime)
ck = sorted(run.glob("checkpoint-*"), key=lambda p: int(p.name.split("-")[-1]), reverse=True)[0]
state = json.loads((ck/"trainer_state.json").read_text())
ev = [d for d in state.get("log_history", []) if "eval_loss" in d]
print(ev[-1] if ev else "no eval yet")

{'epoch': 10.527472527472527, 'eval_loss': 7.096441268920898, 'eval_runtime': 13.6439, 'eval_samples_per_second': 3.518, 'eval_steps_per_second': 0.44, 'step': 600}


snapshot

In [7]:
# Manual snapshot
import pathlib, shutil, time
DRIVE = pathlib.Path("/content/drive/MyDrive/wake2vec")
RUN   = max((DRIVE/"runs").glob("t4_*"), key=lambda p: p.stat().st_mtime)

def latest_ckpt_with_weights(base):
    cands = sorted(base.glob("checkpoint-*"), key=lambda p: int(p.name.split("-")[-1]), reverse=True)
    for ck in cands:
        if (ck/"model.safetensors").exists() or (ck/"pytorch_model.bin").exists():
            return ck
    return None

src = latest_ckpt_with_weights(RUN)
assert src is not None, "No valid checkpoint with weights found yet."
SNAPS = RUN/"snapshots"; SNAPS.mkdir(exist_ok=True, parents=True)
dst = SNAPS/f"snap_{int(time.time())}_{src.name}"
if not dst.exists():
    shutil.copytree(src, dst)
print("[SNAP] Saved", dst)

[SNAP] Saved /content/drive/MyDrive/wake2vec/runs/t4_1762376560/snapshots/snap_1762565503_checkpoint-300


sentry mirror

In [8]:
# newest full checkpoint + metrics
import pathlib, shutil
DRIVE = pathlib.Path("/content/drive/MyDrive/wake2vec")
RUN   = max((DRIVE/"runs").glob("t4_*"), key=lambda p: p.stat().st_mtime)
SENTRY = DRIVE/"sentry_backups"/RUN.name; SENTRY.mkdir(parents=True, exist_ok=True)

def latest_ckpt_with_weights(base):
    cands = sorted(base.glob("checkpoint-*"), key=lambda p: int(p.name.split("-")[-1]), reverse=True)
    for ck in cands:
        if (ck/"model.safetensors").exists() or (ck/"pytorch_model.bin").exists():
            return ck
    return None

ck = latest_ckpt_with_weights(RUN)
if ck is None:
    print("[SENTRY] No full checkpoint yet.")
else:
    dst = SENTRY/ck.name
    if not dst.exists():
        shutil.copytree(ck, dst)
        print(f"[SENTRY] Mirrored {ck.name} → {dst}")
    else:
        print("[SENTRY] Already mirrored:", dst)

# mirror metrics JSONs
mdst = SENTRY/"metrics"; mdst.mkdir(parents=True, exist_ok=True)
for f in (RUN/"metrics").glob("*.json"):
    shutil.copy2(f, mdst/f.name)
print("[SENTRY] Metrics mirrored →", mdst)

[SENTRY] Mirrored checkpoint-300 → /content/drive/MyDrive/wake2vec/sentry_backups/t4_1762376560/checkpoint-300
[SENTRY] Metrics mirrored → /content/drive/MyDrive/wake2vec/sentry_backups/t4_1762376560/metrics


loss and checkpoint

In [9]:
# Heartbeat
import json, pathlib
DRIVE = pathlib.Path("/content/drive/MyDrive/wake2vec")
RUN   = max((DRIVE/"runs").glob("t4_*"), key=lambda p: p.stat().st_mtime)

# loss
mlog = RUN/"metrics"/"phase1_loss_log.json"
if mlog.exists():
    logs = json.loads(mlog.read_text())
    print(f"[LOSS] step={logs[-1]['step']}  loss={float(logs[-1]['loss']):.4f}")
else:
    print("[LOSS] no metrics yet")

# checkpoints
def scan(base):
    rows=[]
    for ck in sorted(base.glob("checkpoint-*"), key=lambda p: int(p.name.split("-")[-1])):
        s=int(ck.name.split("-")[-1])
        rows.append((
            s,
            (ck/"model.safetensors").exists() or (ck/"pytorch_model.bin").exists(),
            (ck/"trainer_state.json").exists(),
            (ck/"optimizer.pt").exists(),
        ))
    return rows

print("\n[RUNS]")
for s,w,t,o in scan(RUN):
    print(f"  {s:>4}  weights={str(w):5}  state={str(t):5}  opt={str(o):5}")

SENTRY = DRIVE/"sentry_backups"/RUN.name
if SENTRY.exists():
    print("\n[SENTRY]")
    for s,w,t,o in scan(SENTRY):
        print(f"  {s:>4}  weights={str(w):5}  state={str(t):5}  opt={str(o):5}")
else:
    print("\n[SENTRY] none")

[LOSS] step=550  loss=3.2604

[RUNS]
   100  weights=True   state=True   opt=True 
   200  weights=True   state=True   opt=True 
   300  weights=True   state=True   opt=True 
   400  weights=False  state=True   opt=True 
   500  weights=False  state=True   opt=True 
   600  weights=False  state=True   opt=True 
   700  weights=False  state=True   opt=True 

[SENTRY]
   300  weights=True   state=True   opt=True 
   400  weights=False  state=True   opt=True 
   500  weights=False  state=True   opt=True 
   600  weights=False  state=False  opt=False
   700  weights=False  state=True   opt=True 


clean drive

In [ ]:
# Light flush to help sync small files
import os, time, pathlib
RUN = max((pathlib.Path("/content/drive/MyDrive/wake2vec")/"runs").glob("t4_*"), key=lambda p: p.stat().st_mtime)
(open(RUN/"_touch.sync","w")).write(str(time.time()))
os.sync()
print("[SYNC] touched + sync hinted")

[SYNC] touched + sync hinted


save if trainer is in scope

In [10]:
try:
    trainer.save_model()
    if hasattr(trainer, "_save_checkpoint"):
        trainer._save_checkpoint(model=trainer.model, trial=None)
    print("[TRAINER] save requested")
except NameError:
    print("[TRAINER] No 'trainer' object in this notebook; use the mirror cell instead.")

[TRAINER] No 'trainer' object in this notebook; use the mirror cell instead.


In [13]:
import json, pathlib
DRIVE = pathlib.Path("/content/drive/MyDrive/wake2vec")
RUN_ID = "t4_1762376560"
RUN = DRIVE/"sentry_backups"/RUN_ID

# Loss heartbeat
mlog = RUN/"metrics"/"phase1_loss_log.json"
if mlog.exists():
    logs = json.loads(mlog.read_text())
    print(f"[LOSS] last step={logs[-1]['step']}  loss={float(logs[-1]['loss']):.4f}")
else:
    print("[LOSS] no metrics yet")

# Checkpoint table
def scan(base):
    rows=[]
    for ck in sorted(base.glob("checkpoint-*"), key=lambda p: int(p.name.split("-")[-1])):
        s=int(ck.name.split("-")[-1])
        rows.append((s,
                     (ck/"model.safetensors").exists() or (ck/"pytorch_model.bin").exists(),
                     (ck/"trainer_state.json").exists(),
                     (ck/"optimizer.pt").exists()))
    return rows

print("\n[RUNS]")
for s,w,t,o in scan(RUN):
    print(f"  {s:>4}  weights={w!s:5}  state={t!s:5}  opt={o!s:5}")

SENTRY = DRIVE/"sentry_backups"/RUN_ID
if SENTRY.exists():
    print("\n[SENTRY]")
    for s,w,t,o in scan(SENTRY):
        print(f"  {s:>4}  weights={w!s:5}  state={t!s:5}  opt={o!s:5}")
else:
    print("\n[SENTRY] none")

[LOSS] last step=550  loss=3.2604

[RUNS]
   300  weights=True   state=True   opt=True 
   400  weights=False  state=True   opt=True 
   500  weights=False  state=True   opt=True 
   600  weights=False  state=False  opt=False
   700  weights=False  state=True   opt=True 

[SENTRY]
   300  weights=True   state=True   opt=True 
   400  weights=False  state=True   opt=True 
   500  weights=False  state=True   opt=True 
   600  weights=False  state=False  opt=False
   700  weights=False  state=True   opt=True 


In [12]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)
print("Drive mounted.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive mounted.


In [16]:
# VERBOSE mirror of latest full checkpoint (+ metrics) to sentry_backups
import pathlib, shutil, time, os

DRIVE = pathlib.Path("/content/drive/MyDrive/wake2vec")
CANDIDATES = [pathlib.Path("/content/runs"), DRIVE/"runs"]

def latest_run():
    runs = []
    for root in CANDIDATES:
        if root.exists():
            for p in root.glob("t4_*"):
                try:
                    runs.append((p.stat().st_mtime, p))
                except FileNotFoundError:
                    pass
    if not runs: return None
    runs.sort(reverse=True)
    return runs[0][1]

def latest_ckpt_with_weights(run):
    if not run: return None
    cks = sorted(run.glob("checkpoint-*"), key=lambda p: int(p.name.split("-")[-1]), reverse=True)
    for ck in cks:
        if (ck/"model.safetensors").exists() or (ck/"pytorch_model.bin").exists():
            return ck
    return None

RUN = latest_run()
print("[INFO] Active run:", RUN if RUN else "none")
CK  = latest_ckpt_with_weights(RUN)
print("[INFO] Latest full ckpt:", CK if CK else "none")

if CK is None:
    print("[SENTRY] No checkpoint with weights yet — will retry after next save.")
else:
    SENTRY = DRIVE/"sentry_backups"/RUN.name
    SENTRY.mkdir(parents=True, exist_ok=True)
    DST = SENTRY/CK.name

    src_mtime = time.ctime(CK.stat().st_mtime)
    print(f"[SENTRY] Source: {CK} (mtime {src_mtime})")
    if DST.exists():
        # check if dest is older/stale by file count or mtime
        src_files = sum(1 for _ in CK.rglob("*"))
        dst_files = sum(1 for _ in DST.rglob("*"))
        print(f"[SENTRY] Already exists: {DST} (files src={src_files} dst={dst_files})")
        if dst_files < src_files:
            print("[SENTRY] Detected partial mirror; refreshing…")
            shutil.rmtree(DST)
            shutil.copytree(CK, DST)
            print("[SENTRY] Re-mirrored:", DST)
        else:
            print("[SENTRY] Mirror up-to-date.")
    else:
        shutil.copytree(CK, DST)
        print("[SENTRY] Mirrored:", DST)

    # Mirror metrics verbosely
    msrc = RUN/"metrics"
    mdst = SENTRY/"metrics"
    mdst.mkdir(parents=True, exist_ok=True)
    copied = 0
    if msrc.exists():
        for f in msrc.glob("*.json"):
            shutil.copy2(f, mdst/f.name)
            copied += 1
    print(f"[SENTRY] Metrics mirrored → {mdst} ({copied} files)")

[INFO] Active run: /content/drive/MyDrive/wake2vec/runs/t4_1762376560
[INFO] Latest full ckpt: /content/drive/MyDrive/wake2vec/runs/t4_1762376560/checkpoint-300
[SENTRY] Source: /content/drive/MyDrive/wake2vec/runs/t4_1762376560/checkpoint-300 (mtime Wed Nov  5 22:45:46 2025)
[SENTRY] Already exists: /content/drive/MyDrive/wake2vec/sentry_backups/t4_1762376560/checkpoint-300 (files src=12 dst=12)
[SENTRY] Mirror up-to-date.
[SENTRY] Metrics mirrored → /content/drive/MyDrive/wake2vec/sentry_backups/t4_1762376560/metrics (1 files)


In [17]:
import pathlib
BASE = pathlib.Path("/content/drive/MyDrive/wake2vec")
RUN = BASE/"runs"/"t4_1762376560"  # adjust if different
SENTRY = BASE/"sentry_backups"/"t4_1762376560"

def audit(root):
    print(f"\n[{root}]")
    for ck in sorted(root.glob("checkpoint-*"), key=lambda p: int(p.name.split("-")[-1])):
        step = int(ck.name.split("-")[-1])
        w = (ck/"model.safetensors").exists() or (ck/"pytorch_model.bin").exists()
        t = (ck/"trainer_state.json").exists()
        o = (ck/"optimizer.pt").exists()
        print(f"{step:>5}  weights={w:<5}  state={t:<5}  opt={o:<5}  → {ck.name}")

if RUN.exists():   audit(RUN)
if SENTRY.exists(): audit(SENTRY)


[/content/drive/MyDrive/wake2vec/runs/t4_1762376560]
  100  weights=1      state=1      opt=1      → checkpoint-100
  200  weights=1      state=1      opt=1      → checkpoint-200
  300  weights=1      state=1      opt=1      → checkpoint-300
  400  weights=0      state=1      opt=1      → checkpoint-400
  500  weights=0      state=1      opt=1      → checkpoint-500
  600  weights=0      state=1      opt=1      → checkpoint-600
  700  weights=0      state=1      opt=1      → checkpoint-700

[/content/drive/MyDrive/wake2vec/sentry_backups/t4_1762376560]
  300  weights=1      state=1      opt=1      → checkpoint-300
  400  weights=0      state=1      opt=1      → checkpoint-400
  500  weights=0      state=1      opt=1      → checkpoint-500
  600  weights=0      state=0      opt=0      → checkpoint-600
  700  weights=0      state=1      opt=1      → checkpoint-700


In [ ]:
# P1 resume
import pathlib, json, shutil, torch, time, os
from datasets import load_from_disk
from transformers import (AutoTokenizer, AutoModelForCausalLM,
                          DataCollatorForLanguageModeling, TrainingArguments, Trainer, TrainerCallback)

DRIVE = pathlib.Path("/content/drive/MyDrive/wake2vec")
RUN_ID = max((DRIVE/"runs").glob("t4_*"), key=lambda p: p.stat().st_mtime).name
LOCAL_RUN = pathlib.Path("/content/runs")/RUN_ID
LOCAL_RUN.mkdir(parents=True, exist_ok=True)
DRIVE_RUN = DRIVE/"runs"/RUN_ID
SENTRY    = DRIVE/"sentry_backups"/RUN_ID

def list_ckpts(base):
    if not base.exists(): return []
    return sorted([p for p in base.glob("checkpoint-*") if p.is_dir()], key=lambda p: int(p.name.split("-")[-1]), reverse=True)

def has_weights(ck):
    return (ck/"model.safetensors").exists() or (ck/"pytorch_model.bin").exists()

# newest w/ weights
cands = list_ckpts(DRIVE_RUN) + list_ckpts(SENTRY)
GOOD = next((p for p in cands if has_weights(p)), None)
assert GOOD is not None, "No valid checkpoint with weights found."
last_step = int(GOOD.name.split("-")[-1])
print(f"[RESUME] {RUN_ID} from {GOOD.name} (last_step={last_step})")

# LOCAL_RUN with checkpoint
if not (LOCAL_RUN/GOOD.name).exists():
    shutil.copytree(GOOD, LOCAL_RUN/GOOD.name)
    print("[LOCAL] seeded", (LOCAL_RUN/GOOD.name))

# tokenizer/model
tok = AutoTokenizer.from_pretrained(str(GOOD), use_fast=True)
if tok.pad_token_id is None: tok.pad_token = tok.eos_token or "</s>"
model = AutoModelForCausalLM.from_pretrained(str(GOOD), torch_dtype=torch.float32, device_map="auto")
model.config.use_cache = False
with torch.no_grad():
    # (re)tie head
    model.get_output_embeddings().weight = model.get_input_embeddings().weight

# data
train_ds = load_from_disk(str(DRIVE/"datasets"/"train_ds"))
valid_ds = load_from_disk(str(DRIVE/"datasets"/"valid_ds")).select(range(min(1000, len(load_from_disk(str(DRIVE/'datasets'/'valid_ds'))))))
dc = DataCollatorForLanguageModeling(tokenizer=tok, mlm=False)

TARGET = 1300

# callbacks
from transformers import TrainerCallback

class EvalEveryNSteps(TrainerCallback):
    def __init__(self, n=200): self.n=n
    def on_step_end(self, args, state, control, **kw):
        if state.global_step and state.global_step % self.n == 0:
            control.should_log = True
            control.should_evaluate = True

class SentryMirror(TrainerCallback):
    def on_save(self, args, state, control, **kw):
        try:
            ck = max(LOCAL_RUN.glob("checkpoint-*"), key=lambda p: int(p.name.split("-")[-1]))
            dst = SENTRY/ck.name
            dst.parent.mkdir(parents=True, exist_ok=True)
            if not dst.exists():
                shutil.copytree(ck, dst)
                print(f"[SENTRY] mirrored {ck.name}")
            # metrics
            msrc = LOCAL_RUN/"metrics"
            mdst = SENTRY/"metrics"; mdst.mkdir(parents=True, exist_ok=True)
            if msrc.exists():
                for f in msrc.glob("*.json"): shutil.copy2(f, mdst/f.name)
            os.sync()
        except Exception as e:
            print("[SENTRY] mirror failed:", e)

# Embedding-only snapshot
class EmbeddingSnap(TrainerCallback):
    def __init__(self, every=50):
        self.every = every
        (DRIVE/"emb_snaps"/RUN_ID).mkdir(parents=True, exist_ok=True)
    def on_step_end(self, args, state, control, **kw):
        if state.global_step and state.global_step % self.every == 0:
            try:
                emb = model.get_input_embeddings().weight.detach().cpu()
                path = DRIVE/"emb_snaps"/RUN_ID/f"emb_step{int(state.global_step):04d}.pt"
                torch.save(emb, path)
                # tiny heartbeat json
                hb = {"step": int(state.global_step), "ts": time.time(), "rows": int(emb.size(0)), "dim": int(emb.size(1))}
                (DRIVE/"emb_snaps"/RUN_ID/"heartbeat.json").write_text(json.dumps(hb, indent=2))
                print(f"[SNAP] embeddings → {path.name}")
            except Exception as e:
                print("[SNAP] failed:", e)

# dynamic save_steps
save_steps = 50 if last_step < 700 else 100

from transformers import TrainingArguments
args = TrainingArguments(
    output_dir=str(LOCAL_RUN),
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    max_steps=TARGET,
    learning_rate=5e-4,
    warmup_ratio=0.0,
    optim="adafactor",
    logging_steps=50,
    save_strategy="steps",
    save_steps=save_steps,
    save_total_limit=15,
    gradient_checkpointing=True,
    fp16=False, bf16=False,
    report_to=["none"],
    max_grad_norm=1.0,
)

from transformers import Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=valid_ds,
    data_collator=dc,
    callbacks=[EvalEveryNSteps(200), SentryMirror(), EmbeddingSnap(50)],
)

trainer.train(resume_from_checkpoint=str(LOCAL_RUN/GOOD.name))
# final save
trainer.save_model(str(LOCAL_RUN/"checkpoint-final")); tok.save_pretrained(str(LOCAL_RUN/"checkpoint-final"))
# mirror the final
try:
    dst = SENTRY/"checkpoint-final"
    if dst.exists(): shutil.rmtree(dst)
    shutil.copytree(LOCAL_RUN/"checkpoint-final", dst)
    print("[SENTRY] mirrored checkpoint-final")
except Exception as e:
    print("[SENTRY] final mirror failed:", e)

print("[DONE] Reached", TARGET)